## Exploring metadata for dashboard

In [9]:
from dotenv import load_dotenv
import os 
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)

result = response.json()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [10]:
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [11]:
example_trip["LegList"]["Leg"][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration'])

In [12]:
stops = example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
stops

[{'name': 'Göteborg Centralstation',
  'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
  'extId': '740000002',
  'routeIdx': 0,
  'lon': 11.973479,
  'lat': 57.708895,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '20:40:00',
  'depDate': '2025-01-20',
  'depDir': 'Malmö Hyllie station',
  'minimumChangeDuration': 'PT15M'},
 {'name': 'Mölndal station',
  'id': 'A=1@O=Mölndal station@X=12018722@Y=57655958@U=1@L=740000315@',
  'extId': '740000315',
  'routeIdx': 1,
  'lon': 12.018722,
  'lat': 57.655958,
  'depTime': '20:48:00',
  'depDate': '2025-01-20',
  'arrTime': '20:48:00',
  'arrDate': '2025-01-20',
  'minimumChangeDuration': 'PT9M'},
 {'name': 'Kungsbacka station',
  'id': 'A=1@O=Kungsbacka station@X=12079614@Y=57490556@U=1@L=740000161@',
  'extId': '740000161',
  'routeIdx': 2,
  'lon': 12.079614,
  'lat': 57.490556,
  'depTime': '20:59:00',
  'depDate': '2025-01-20',
  'arrTime': '20:59:00',
  'arrDate': '2025-01-20',
  'minimumChangeDuration': '

In [33]:
for stop in stops:
    print(f"Hållplats: {stop.get('name')}")
    print(f"Ankomsttid: {stop.get('arrTime')}")
    print(f"Avgångstid: {stop.get('depTime')}")
    print(f"Koordinater: Lat {stop.get('lat')}, Lon {stop.get('lon')}")
    print("-" * 30)

Hållplats: Göteborg Centralstation
Ankomsttid: None
Avgångstid: 20:40:00
Koordinater: Lat 57.708895, Lon 11.973479
------------------------------
Hållplats: Mölndal station
Ankomsttid: 20:48:00
Avgångstid: 20:48:00
Koordinater: Lat 57.655958, Lon 12.018722
------------------------------
Hållplats: Kungsbacka station
Ankomsttid: 20:59:00
Avgångstid: 20:59:00
Koordinater: Lat 57.490556, Lon 12.079614
------------------------------
Hållplats: Varberg station
Ankomsttid: 21:25:00
Avgångstid: 21:29:00
Koordinater: Lat 57.109378, Lon 12.248081
------------------------------
Hållplats: Falkenberg station
Ankomsttid: 21:45:00
Avgångstid: 21:45:00
Koordinater: Lat 56.919768, Lon 12.507555
------------------------------
Hållplats: Halmstad Centralstation
Ankomsttid: 22:03:00
Avgångstid: 22:13:00
Koordinater: Lat 56.669275, Lon 12.86484
------------------------------
Hållplats: Laholm station
Ankomsttid: 22:25:00
Avgångstid: 22:25:00
Koordinater: Lat 56.501905, Lon 12.999606
---------------------

In [34]:
legs = example_trip["LegList"]["Leg"]
for leg in legs:
    print(f"Transport: {leg.get('name')} ({leg.get('category')})")
    print(f"Från: {leg['Origin']['name']} ({leg['Origin']['time']})")
    print(f"Till: {leg['Destination']['name']} ({leg['Destination']['time']})")
    print("-" * 30)

Transport: Regional Tåg 11159 (JRE)
Från: Göteborg Centralstation (20:40:00)
Till: Malmö Centralstation (23:48:00)
------------------------------


In [35]:
import folium

start_lat = float(legs[0]['Origin']['lat'])
start_lon = float(legs[0]['Origin']['lon'])

m = folium.Map(location=[start_lat, start_lon], zoom_start=7)

for leg in legs:
    folium.Marker(
        location=[float(leg['Origin']['lat']), float(leg['Origin']['lon'])],
        popup=f"Start: {leg['Origin']['name']} ({leg['Origin']['time']})",
        icon=folium.Icon(color="green")
    ).add_to(m)

    if 'Stops' in leg and leg['Stops']:
        stops = leg['Stops']['Stop']
        for stop in stops:
            folium.Marker(
                location=[float(stop['lat']), float(stop['lon'])],
                popup=f"Hållplats: {stop['name']} ({stop.get('depTime', 'N/A')})",
                icon=folium.Icon(color="blue", icon="info-sign")
            ).add_to(m)

    folium.Marker(
        location=[float(leg['Destination']['lat']), float(leg['Destination']['lon'])],
        popup=f"Slut: {leg['Destination']['name']} ({leg['Destination']['time']})",
        icon=folium.Icon(color="red")
    ).add_to(m)

m

## Total travel time and number of stops in between

In [43]:
import pandas as pd

time_data = []
for leg in legs:
    origin_time = pd.to_datetime(leg['Origin']['time'])
    destination_time = pd.to_datetime(leg['Destination']['time'])
    duration = destination_time - origin_time
    
    stops = leg.get('Stops', {}).get('Stop', [])
    num_stops_between = max(0, len(stops) - 2)
    
    time_data.append({
        "Transport": leg.get("name"),
        "Start": origin_time,
        "Slut": destination_time,
        "Total tid": duration,
        "Antal stopp emellan": num_stops_between
    })

df_time = pd.DataFrame(time_data)
print(df_time)

            Transport               Start                Slut       Total tid  \
0  Regional Tåg 11159 2025-01-21 20:40:00 2025-01-21 23:48:00 0 days 03:08:00   

   Antal stopp emellan  
0                   11  


EDA för uppgift 4 - reseplaneraren: karta med punkter för varje station mellan ursprung och slutdestination (Göteborg Centralstation - Malmö Centralstation), total tid för resan och antal stopp emellan.